In [1]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import requests
import urllib.request, json 
import re
import time
from datetime import date

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
    
def download_pdf(url,path,filename):
    try:
        
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup
def download_pdf_unmasked(url,path,filename):
    
        
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)

In [19]:
source = "https://selfhelp.nvcourts.gov/forms?enter=1"
BASE  = "https://selfhelp.nvcourts.gov"

soup = connect_to_page(source)

navigation = soup.find("ul",class_="nav menusub")

cat_list = navigation.find_all("li")
cat_hrefs = []
group_names = []

for cat in cat_list:
    cat_href = cat.find('a',href=True)['href']
    cat_hrefs.append(BASE+cat_href)
    group_names.append(cat.text)
    print(cat.text)
    print(BASE+cat_href)

Divorce Forms
https://selfhelp.nvcourts.gov/forms/divorce-forms
Custody Paternity & Child Support Forms
https://selfhelp.nvcourts.gov/forms/custody-paternity-child-support-forms
Legal Separation Forms
https://selfhelp.nvcourts.gov/forms/legal-separation-forms
Annulment Forms
https://selfhelp.nvcourts.gov/forms/annulment-forms
Motions & Oppositions
https://selfhelp.nvcourts.gov/forms/motions-oppositions
Name Change Forms
https://selfhelp.nvcourts.gov/forms/name-change-forms
Guardianship Forms
https://selfhelp.nvcourts.gov/forms/guardianship-forms
Adoption & Termination of Parental Rights Forms
https://selfhelp.nvcourts.gov/forms/adoption-termination-of-parental-rights-forms
Temporary Protection Orders
https://selfhelp.nvcourts.gov/forms/temporary-protection-orders
Miscellaneous Forms
https://selfhelp.nvcourts.gov/forms/miscellaneous-forms


In [ ]:
today = date.today().strftime("%Y-%m-%d")
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded",])
form_dest = "NV/NV_AllForms/"
jur = "NV"
pdfs = {}

for i in range(len(cat_hrefs)):
    soup = connect_to_page(cat_hrefs[i])
    pdf_refs = soup.find_all("a", href=True)
    group = group_names[i]
    print(group)

    for ref in pdf_refs:
        if ".pdf" in ref['href']:
            filename = ref['href'].split('/')[-1]
            if filename not in pdfs:
                print(filename)
                url = BASE+ref['href']
                id = hashme(url)
                title = re.sub(r"\([^()]*\)","",ref.text)
                print(title)
                status = download_pdf(url,form_dest,id+".pdf")
                if status=="error":
                    print("####################")
                    print("ERROR:", url)
                    print("####################")

                files_df = files_df.append(pd.DataFrame([[id,jur,source,title,group,url,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]))
files_df.to_csv("NV-V1.csv",index=False, encoding="utf-8")
            
            
